In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import scipy.signal as ss

import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
EEG_h = np.load('/home/e/etayhay/frankm/Mazza2023/data/simulations_processed/EEG_h.npy',allow_pickle=True)
EEG_m = np.load('/home/e/etayhay/frankm/Mazza2023/data/simulations_processed/EEG_m.npy',allow_pickle=True)

end = int(round(100/.333333)) #only if window is 3s

EEG_h_ps = np.array(EEG_h.item(0)['mean_psd']['ps_wel_bm'][:end])
EEG_h_sd = np.array(EEG_h.item(0)['mean_psd']['sd_wel'][:end])
EEG_h_CI_upper = np.array(EEG_h.item(0)['mean_psd']['ps_wel_upper'][:end])
EEG_h_CI_lower = np.array(EEG_h.item(0)['mean_psd']['ps_wel_lower'][:end])
EEG_h_freq = np.array(EEG_h.item(0)['mean_psd']['freq_wel'][:end])
EEG_m_ps = np.array(EEG_m.item(0)['mean_psd']['ps_wel_bm'][:end])
EEG_m_sd = np.array(EEG_m.item(0)['mean_psd']['sd_wel'][:end])
EEG_m_CI_upper = np.array(EEG_m.item(0)['mean_psd']['ps_wel_upper'][:end])
EEG_m_CI_lower = np.array(EEG_m.item(0)['mean_psd']['ps_wel_lower'][:end])
EEG_m_freq = np.array(EEG_m.item(0)['mean_psd']['freq_wel'][:end])


PSDclr = {'healthy':'k', 'mdd':(0.8, 0.0, 0.45)}
labelfont = {'family':'arial','weight':'bold','size':14}
sampling_rate = (1/0.025)*1000
nperseg_PSD = int(sampling_rate*.5)#.5s window for spectrogram calc
noverlap_PSD=nperseg_PSD*.8

In [ ]:
def plot_psd(fig,plotspecs,freq,psd1,sd1,CIupper1,CIlower1,color1,psd2=False,sd2=False,CIupper2=False,CIlower2=False,color2=False,add_inset=False):
    psd2_true = True if type(psd2) == np.ndarray else None
    ax = fig.add_axes(plotspecs)
    ax.plot(freq,psd1,color=PSDclr[color1])
    ax.plot(freq,psd2,color=PSDclr[color2]) if psd2_true else None
    ax.fill_between(freq,CIupper1,CIlower1,color=PSDclr[color1],alpha=std_alpha)
    ax.fill_between(freq,CIupper2,CIlower2,color=PSDclr[color2],alpha=std_alpha) if psd2_true else None
    if add_inset:
        inset = ax.inset_axes([.6,.6,.4,.4])
        inset.plot(freq,psd1,color=PSDclr[color1])
        inset.plot(freq,psd2,color=PSDclr[color2]) if psd2_true else None
        ylims = [min(psd1[:freq.size]),max(psd1[:freq.size])*1.1] if not psd2_true else [min(psd1[:freq.size]),max(psd2[:freq.size])*1.1]
        inset.set_ylim(ylims)
        inset.fill_between(freq,CIupper1,CIlower1,color=PSDclr[color1],alpha=std_alpha)
        inset.fill_between(freq,CIupper2,CIlower2,color=PSDclr[color2],alpha=std_alpha) if psd2_true else None

        return ax,inset
    if not add_inset:
        return ax

In [ ]:
#==============================================
# FIGURE 2
#==============================================

run =1000

fig = plt.figure(figsize=(10,3),frameon=False)
gsize = (30,1)
x = -.14

color = 'healthy' if run<2000 else 'mdd'
EEG = EEG_h.item(0)[str(run)]['ts_raw'] if run<2000 else EEG_m.item(0)[str(run)]['ts_raw'] #spectro & signal

std_alpha=.15
# shift = 3
    
c_EEG_PSD,c_EEG_inset = plot_psd(fig,[0,.0,.43,1],EEG_h_freq,EEG_h_ps,EEG_h_sd,EEG_h_CI_upper,EEG_h_CI_lower,'healthy',add_inset=True) if run<2000 else plot_psd(gsize,(0,0),19,4,EEG_m_freq,EEG_m_ps,EEG_m_sd,EEG_m_CI_upper,EEG_m_CI_lower,'mdd',add_inset=True)



axes = [c_EEG_PSD]
insets = [c_EEG_inset]
[ax.set_ylabel(r' Spectral Power $\left( \frac{ V^{2} }{Hz} \right)$') for ax in axes]
#[ax.set_ylabel('Relative Power a.u.') for ax in axes]
[[ax.axvline(i,0,1, linestyle=':',color='k') for ax in axes] for i in [4,8,12]]
[ax.spines["top"].set_visible(False) for ax in axes+insets]
[ax.spines["right"].set_visible(False) for ax in axes+insets]
[ax.set_xscale('log') for ax in insets]
[ax.set_yscale('log') for ax in insets]
[ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter()) for ax in insets]
[ax.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter()) for ax in insets]
[ax.set_xticks([1, 10, 50, 100]) for ax in insets]
[ax.set_xticks([5,10,15,20,25,30]) for ax in axes]

[ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0)) for ax in axes]
[ax.set_yticks([]) for ax in insets]
[ax.set_yticklabels((),visible=False) for ax in insets]
c_EEG_inset.set_yticks([])
c_EEG_inset.set_yticklabels((),visible=False)
c_EEG_inset.tick_params(axis='both', which='major')
c_EEG_PSD.set_xlabel('Frequency (Hz)')
c_EEG_PSD.tick_params(axis='both', which='major')

[ax.set_xlim([2,30]) for ax in axes]

c_EEG_inset.tick_params(axis = 'x',which = 'both',labelbottom=False)
c_EEG_inset.set_xlim(4,30)

fig.text(x=.272,y=.527,s='5')
fig.text(x=.33,y=.527,s='10')
fig.text(x=.42,y=.527,s='30')

xs = [5.9,9.9,19.9]
labels = [r'$\theta$',r'$\alpha$',r'$\beta$']
[c_EEG_PSD.annotate(xy=(x,0.01e-14),s=label) for x,label in zip(xs,labels)]

#[c_EEG_inset.annotate(xy=(x,1.2e-15),s=t,fontsize=10) for x,t in zip([5,8.5,22],labels)]


[ax.set_xlim([4,30]) for ax in insets]

offset = 1.9e-14
AP_example = (10**offset) * (1 / (EEG_h_freq[12:]**1.2))*9e-14
c_EEG_inset.plot(EEG_h_freq[12:], AP_example, ls = '--',zorder =0, color = 'k')
c_EEG_inset.set_ylim(1e-15, 2e-14)

c_EEG_inset.annotate(xy=(8,.3e-14),s=r'$\frac{1}{f}$', size=16)








EEG_sp = fig.add_axes([.52,0,.6,1])


fs = 40000
nperseg_PSD = int(fs*.5)
nfft=int(fs*3)
noverlap = int(nperseg_PSD*.6)

freq,time,Sxx = ss.spectrogram(EEG_h.item(0)[str(run)]['ts_raw'][80000:80000*17],
                               fs=fs,
                               window='hann',
                               nperseg=nperseg_PSD,
                               nfft=nfft,
                               noverlap=noverlap_PSD)

# gives spectrogram with 20000 freqs,slows down plotting so only use up to 30Hz
e = np.where(freq==31)[0][0]

freq=freq[:e]
Sxx = Sxx[:e]

levels = np.linspace(-15, -12.8, 50)

cf = EEG_sp.contourf(time,freq,np.log10(Sxx),levels=levels, extend='min',ls=None)

cbar = fig.colorbar(cf, ticks = np.array([-13, -13.5, -14, -14.5, -15]))


cbar.set_label(r' Power $\frac{dB}{Hz}$', rotation=270,labelpad=25)

EEG_sp.set_ylim(0,30)
EEG_sp.set_xlim(10,14)
EEG_sp.set_ylabel('Frequency (Hz)')
EEG_sp.set_xlabel('Time (s)')

c_EEG_PSD.set_title('A', x=-.1, y=1.06,**labelfont)
EEG_sp.set_title('B', x=-.1,y=1.06, **labelfont)



fig.savefig('/home/e/etayhay/frankm/Mazza2023/figures/figoutput/Figure2.jpeg', facecolor='white', edgecolor='none',bbox_inches = "tight",dpi=600)
